In [1]:
%load_ext autoreload
%autoreload 2
!hostname
!pwd
import os, sys
print(sys.executable)

fish-gcp005
/home/akarshkumar0101/nca-alife
/home/akarshkumar0101/miniconda3/envs/nca-alife-torch/bin/python


In [2]:
import os, sys, glob, pickle
from functools import partial

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_theme()

from tqdm.auto import tqdm
from einops import rearrange, reduce, repeat

In [3]:
# import experiment_utils
import time
import subprocess

# Launch Experiment (on this node)

In [4]:
header = '\n'.join([
    '#!/bin/bash',
    
    # 'source ~/.virtualenvs/nca-alife-torch/bin/activate'
    
    'source ~/miniconda3/etc/profile.d/conda.sh',
    'conda activate nca-alife-torch',
    
    'cd ~/nca-alife/src'
])


gpus = [4, 5, 6, 7]
log_dir = os.path.expanduser(f'~/nca-alife-data/logs/{time.strftime("%Y_%m_%d_%H_%M_%S")}')
exp_sh = f'./src/experiments/main.sh'
with open(exp_sh, 'r') as f:
    commands = [c for c in f.read().split('\n') if c]
print(f"{exp_sh} has {len(commands)} commands: ")
print('\n'.join(commands[:3]), '\n...')
print(f"Logging to {log_dir}")

./src/experiments/main.sh has 24 commands: 
python nca_multi_prompt_clip.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/nca_exp1/seed=0_i1=0_perception='gradient'_img_size=32"  --dtype="float32" --device="cuda:0" --img_size=32  --d_state=16 --perception="gradient" --kernel_size=3 --init_state="point" --padding_mode="zeros" --dt=0.01 --dropout=0.5 --target_img_path=None --prompt=";a very skinny cat;a skinny cat;a slightly skinny cat;a normal cat;a slightly fat cat;a fat cat;a very fat cat"                                                                                                                                                                                                                                                       --augs="crop+pers" --aug_crop_scale=1.0 --clip_model="clip-vit-base-patch32" --coef_alignment=1.0 --coef_softmax=0.03 --coef_novelty=0.0 --coef_temporal=0.0 --rollout_steps=1024 --bptt_steps=16 --pool_size=1024 --bs=8 --lr=0.0003 --n_iters=200000

In [5]:
print(log_dir)

/home/akarshkumar0101/nca-alife-data/logs/2024_07_04_05_24_11


In [6]:
!mkdir -p {log_dir}
!rm {os.path.dirname(log_dir)+'/latest'}
!ln -sf {log_dir} {os.path.dirname(log_dir)+'/latest'}
for i_gpu, gpu in enumerate(gpus):
    job_ids = np.arange(len(commands))[i_gpu::len(gpus)]
    cvd = f'export CUDA_VISIBLE_DEVICES={gpu}'
    txt = f"{header}\n{cvd}\n\n"
    for job_id in job_ids:
        txt += f"touch {log_dir}/job_{job_id:05d}.status\n"
        txt += f"{commands[job_id]} &> {log_dir}/job_{job_id:05d}.out\n"
        txt += f"echo $? > {log_dir}/job_{job_id:05d}.status\n\n"
    with open(f"{log_dir}/gpu_{gpu}.sh", "w") as f:
        f.write(txt)

In [7]:
print('\n'.join(txt.split('\n')[:12]))

#!/bin/bash
source ~/miniconda3/etc/profile.d/conda.sh
conda activate nca-alife-torch
cd ~/nca-alife/src
export CUDA_VISIBLE_DEVICES=7

touch /home/akarshkumar0101/nca-alife-data/logs/2024_07_04_05_24_11/job_00003.status
python nca_multi_prompt_clip.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/nca_exp1/seed=0_i1=0_perception='gradient'_img_size=64"  --dtype="float32" --device="cuda:0" --img_size=64  --d_state=16 --perception="gradient" --kernel_size=3 --init_state="point" --padding_mode="zeros" --dt=0.01 --dropout=0.5 --target_img_path=None --prompt=";a very skinny cat;a skinny cat;a slightly skinny cat;a normal cat;a slightly fat cat;a fat cat;a very fat cat"                                                                                                                                                                                                                                                       --augs="crop+pers" --aug_crop_scale=1.0 --clip_model="clip-vit-base-p

In [8]:
for i_gpu, gpu in enumerate(gpus):
    command = f"bash {log_dir}/gpu_{gpu}.sh"
    print(command)
    process = subprocess.Popen(command, shell=True)

bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_04_05_24_11/gpu_4.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_04_05_24_11/gpu_5.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_04_05_24_11/gpu_6.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_04_05_24_11/gpu_7.sh


# Track Experiment

In [9]:
a = []
for job_id in range(len(commands)):
    try:
        with open(f'{log_dir}/job_{job_id:05d}.status', 'r') as f:
            a.append(int(f.read().strip()))
    except Exception as e:
        print(job_id, e)
        a.append(-1)
a = np.array(a)

0 invalid literal for int() with base 10: ''
1 invalid literal for int() with base 10: ''
2 invalid literal for int() with base 10: ''
3 invalid literal for int() with base 10: ''
4 [Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_07_04_05_24_11/job_00004.status'
5 [Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_07_04_05_24_11/job_00005.status'
6 [Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_07_04_05_24_11/job_00006.status'
7 [Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_07_04_05_24_11/job_00007.status'
8 [Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_07_04_05_24_11/job_00008.status'
9 [Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_07_04_05_24_11/job_00009.status'
10 [Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_07_04_05_24_

In [10]:
a

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1])

In [11]:
np.array(a).mean()

-1.0

In [12]:
len(commands)

24

In [17]:
for job_id in range(len(commands)):
    print(job_id)
    try:
        with open(f"{log_dir}/job_{job_id:05d}.out", "r") as f:
            print(f.readlines()[-1].strip())
    except Exception as e:
        print(e)

0
100%|██████████| 200000/200000 [1:30:17<00:00, 36.92it/s, loss=-0.287]
1
100%|██████████| 200000/200000 [1:25:55<00:00, 38.79it/s, loss=-0.308]
2
100%|██████████| 200000/200000 [1:23:17<00:00, 40.02it/s, loss=-0.333]
3
100%|██████████| 200000/200000 [1:31:55<00:00, 36.26it/s, loss=-0.275]
4
100%|██████████| 200000/200000 [1:24:02<00:00, 39.67it/s, loss=-0.262]
5
100%|██████████| 200000/200000 [1:34:16<00:00, 35.36it/s, loss=-0.304]
6
100%|██████████| 200000/200000 [1:53:09<00:00, 29.46it/s, loss=-0.266]
7
100%|██████████| 200000/200000 [1:56:05<00:00, 28.71it/s, loss=-0.296]
8
100%|██████████| 200000/200000 [1:57:43<00:00, 28.32it/s, loss=-0.321]
9
100%|██████████| 200000/200000 [3:56:14<00:00, 14.11it/s, loss=-0.259]
10
100%|██████████| 200000/200000 [4:04:46<00:00, 13.62it/s, loss=-0.207]
11
100%|██████████| 200000/200000 [4:13:32<00:00, 13.15it/s, loss=-0.259]
12
100%|██████████| 200000/200000 [1:31:00<00:00, 36.63it/s, loss=-0.29]
13
100%|██████████| 200000/200000 [1:20:55<00:00,